In [10]:
import json
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.utils import pad_sequences
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.layers import Masking
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout
features_d1 = []
labels_d1 = []
# Open file for reading
with open('domain1_train.json', 'r') as f:
    for line in f:
        # Parse the JSON line into a Python dictionary
        obj = json.loads(line)
        features_d1.append(obj['text'])
        labels_d1.append(obj['label'])
features_d2 = []
labels_d2 = []
# Open file for reading
with open('domain2_train.json', 'r') as f:
    for line in f:
        # Parse the JSON line into a Python dictionary
        obj = json.loads(line)
        features_d2.append(obj['text'])
        labels_d2.append(obj['label'])
features = features_d1 + features_d2
labels = labels_d1 + labels_d2

In [11]:

features = pad_sequences(features, padding='pre', maxlen=150, value=-1)
labels = to_categorical(labels, num_classes=2)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [25]:
len(X_train)

27520

In [33]:
vocab_size = 5000  # Adjust based on your actual vocabulary size
embedding_dim = 128
sequence_length = 150
n_classes = 2  # Binary classification

from keras.models import Sequential
from tensorflow.keras.layers import Embedding, Masking, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization, Flatten, AdditiveAttention, GlobalAveragePooling1D
from keras.regularizers import l1, l2
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Include early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=sequence_length),
    Masking(mask_value=-1),  # Usually 0 is used for padding
    Conv1D(64, 5, activation='relu', kernel_regularizer=l2(0.001)),  # Reduced number of filters and regularization
    BatchNormalization(),
    MaxPooling1D(5),
    Dropout(0.5),  # Increased dropout rate
    Bidirectional(LSTM(32, recurrent_regularizer=l2(0.001), kernel_regularizer=l2(0.001))),  # Reduced LSTM units
    Dropout(0.5),  # Increased dropout rate
    Dense(n_classes, activation='sigmoid', kernel_regularizer=l1(0.001))  # Reduced regularization and activation changed to sigmoid for binary classification
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [35]:
model.fit(X_train, y_train, validation_split=0.2, epochs=50, callbacks=[early_stopping])

Epoch 1/50
688/688 [==============================] - 19s 22ms/step - loss: 0.5077 - accuracy: 0.8824 - val_loss: 0.3966 - val_accuracy: 0.8975
Epoch 2/50
688/688 [==============================] - 11s 16ms/step - loss: 0.3410 - accuracy: 0.9090 - val_loss: 0.3436 - val_accuracy: 0.9026
Epoch 3/50
688/688 [==============================] - 10s 14ms/step - loss: 0.2905 - accuracy: 0.9190 - val_loss: 0.3543 - val_accuracy: 0.8972
Epoch 4/50
688/688 [==============================] - 10s 15ms/step - loss: 0.2561 - accuracy: 0.9276 - val_loss: 0.3621 - val_accuracy: 0.8937
Epoch 5/50
688/688 [==============================] - 11s 16ms/step - loss: 0.2254 - accuracy: 0.9394 - val_loss: 0.3780 - val_accuracy: 0.9006


In [36]:
test_ids = []
test_texts = []
# Open file for reading
with open('test_set.json', 'r') as f:
    for line in f:
        # Parse the JSON line into a Python dictionary
        obj = json.loads(line)
        test_ids.append(obj['id'])
        test_texts.append(obj['text'])